In [1]:
import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:

X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)



In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [11]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [13]:
from scipy.spatial import distance
from sklearn.metrics import pairwise

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, kernel = 'polynomial', parameters = None):

    rank_accuracies = []
    AP = []

    # Break condition for testing
    #q = 0

    for query, camId_q, y_q in zip(X_query, camId_query, y_query):
        q_g_dists = []
        y_valid = []
        for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
            if ((camId_q == camId_g) and (y_q == y_g)):
                continue
            else:
                dist = pairwise.kernel_metrics(query, gallery)
                if kernel == 'polynomial':
                    dist = pairwise.polynomial_kernel(query, gallery)
                elif kernel == 'gaussian_rbf':
                    dist = pairwise.rbf_kernel(query, gallery)
                elif kernel == 'sigmoid':
                    dist = pairwise.sigmoid_kernel(query, gallery)
                elif kernel == 'chi2':
                    dist = pairwise.chi2_kernel(query, gallery)
                elif kernel == 'mahalanobis':
                    dist = distance.mahalanobis(query, gallery, parameters)
                else:
                    raise NameError('Specified metric not supported')
                print (dist)
                q_g_dists.append(dist)
                y_valid.append(y_g)
    
        tot_label_occur = y_valid.count(y_q)
    
        q_g_dists = np.array(q_g_dists)
        y_valid = np.array(y_valid)
    
        _indexes = np.argsort(q_g_dists)
    
        # Sorted distances and labels
        q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
        AP_, rank_A = get_acc_score(y_valid, y_q, tot_label_occur)
    
        AP.append(AP_)
        
        rank_accuracies.append(rank_A)
    
        #if q  > 5:
        #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))
    
    return rank_accuracies, mAP

    
        
    

In [14]:
rank_accuracies_l = []
mAP_l = []
metric_l = []

In [15]:
# Baseline Euclidian
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       kernel ='additive_chi2',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Euclidian')


TypeError: kernel_metrics() got an unexpected keyword argument 'metric'

In [17]:
# Square Euclidian
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = 'seuclidean',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Square Euclidian')

Accuracies by Rank:
Rank  1  = 47.00% 	 Rank  2  = 54.57% 	 Rank  3  = 59.64% 	 Rank  4  = 63.93% 	 Rank  5  = 66.86%
Rank  6  = 69.29% 	 Rank  7  = 71.14% 	 Rank  8  = 72.36% 	 Rank  9  = 73.71% 	 Rank  10  = 74.93%
Rank  11  = 75.86% 	 Rank  12  = 76.79% 	 Rank  13  = 77.71% 	 Rank  14  = 78.50% 	 Rank  15  = 79.07%
Rank  16  = 79.86% 	 Rank  17  = 80.64% 	 Rank  18  = 81.57% 	 Rank  19  = 82.29% 	 Rank  20  = 83.21%
Rank  21  = 83.50% 	 Rank  22  = 83.71% 	 Rank  23  = 84.00% 	 Rank  24  = 84.29% 	 Rank  25  = 84.79%
Rank  26  = 85.29% 	 Rank  27  = 85.64% 	 Rank  28  = 85.93% 	 Rank  29  = 86.07% 	 Rank  30  = 86.36%
mAP = 46.57%


In [ ]:
# Standardized Euclidian
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = 'sqeuclidean',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Standardized Euclidian')

In [18]:
#Manhattan Distance

rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = 'minkowski',
                                       parameters = 1)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Manhattan')

Accuracies by Rank:
Rank  1  = 47.21% 	 Rank  2  = 54.79% 	 Rank  3  = 59.86% 	 Rank  4  = 63.71% 	 Rank  5  = 66.14%
Rank  6  = 68.79% 	 Rank  7  = 70.29% 	 Rank  8  = 72.07% 	 Rank  9  = 73.79% 	 Rank  10  = 75.07%
Rank  11  = 75.93% 	 Rank  12  = 77.36% 	 Rank  13  = 78.43% 	 Rank  14  = 79.00% 	 Rank  15  = 79.79%
Rank  16  = 80.00% 	 Rank  17  = 80.57% 	 Rank  18  = 81.21% 	 Rank  19  = 82.29% 	 Rank  20  = 82.64%
Rank  21  = 82.93% 	 Rank  22  = 83.43% 	 Rank  23  = 83.86% 	 Rank  24  = 84.21% 	 Rank  25  = 84.50%
Rank  26  = 84.93% 	 Rank  27  = 85.36% 	 Rank  28  = 85.64% 	 Rank  29  = 85.93% 	 Rank  30  = 86.00%
mAP = 46.78%


In [ ]:
# Chebyshev - L_infinity

rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='chebyshev',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Chebyshev')

In [ ]:
# Braycurtis

rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='braycurtis',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Bray Curtis')

In [ ]:
# Canberra

rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='canberra',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Canberra')

In [19]:
# Cosine

rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='cosine',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Cosine')

Accuracies by Rank:
Rank  1  = 47.57% 	 Rank  2  = 55.07% 	 Rank  3  = 60.50% 	 Rank  4  = 63.50% 	 Rank  5  = 67.00%
Rank  6  = 69.00% 	 Rank  7  = 71.07% 	 Rank  8  = 72.57% 	 Rank  9  = 74.21% 	 Rank  10  = 75.07%
Rank  11  = 75.93% 	 Rank  12  = 76.71% 	 Rank  13  = 77.36% 	 Rank  14  = 78.14% 	 Rank  15  = 79.43%
Rank  16  = 80.14% 	 Rank  17  = 81.00% 	 Rank  18  = 81.71% 	 Rank  19  = 82.43% 	 Rank  20  = 82.71%
Rank  21  = 83.00% 	 Rank  22  = 83.71% 	 Rank  23  = 83.86% 	 Rank  24  = 84.43% 	 Rank  25  = 85.00%
Rank  26  = 85.07% 	 Rank  27  = 85.36% 	 Rank  28  = 85.57% 	 Rank  29  = 85.86% 	 Rank  30  = 85.93%
mAP = 47.21%


In [20]:
# Correlation

rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='correlation',
                                       parameters = None)

rank_accuracies_l.append(rank_accuracies)
mAP_l.append(mAP)
metric_l.append('Correlation')

Accuracies by Rank:
Rank  1  = 46.93% 	 Rank  2  = 54.64% 	 Rank  3  = 59.71% 	 Rank  4  = 63.36% 	 Rank  5  = 66.36%
Rank  6  = 68.07% 	 Rank  7  = 69.79% 	 Rank  8  = 71.21% 	 Rank  9  = 73.07% 	 Rank  10  = 74.29%
Rank  11  = 75.07% 	 Rank  12  = 76.14% 	 Rank  13  = 76.93% 	 Rank  14  = 77.64% 	 Rank  15  = 78.29%
Rank  16  = 79.14% 	 Rank  17  = 80.07% 	 Rank  18  = 80.50% 	 Rank  19  = 81.57% 	 Rank  20  = 82.21%
Rank  21  = 82.79% 	 Rank  22  = 83.50% 	 Rank  23  = 83.64% 	 Rank  24  = 84.07% 	 Rank  25  = 84.21%
Rank  26  = 84.57% 	 Rank  27  = 85.14% 	 Rank  28  = 85.21% 	 Rank  29  = 85.64% 	 Rank  30  = 86.07%
mAP = 46.73%


In [ ]:
plt.figure(figsize=(8.0, 6.0))
color_list = ['green', 'blue', 'red', 'purple', 'orange', 'magenta', 'cyan', 'black', 'indianred', 'lightseagreen', 'gold', 'lightgreen']
for i in range(len(metric_l)):
    plt.plot(np.arange(1, 31), rank_accuracies_l[i], color=color_list[i], linestyle='dashed', label='Metric: '+ metric_l[i])

plt.title('CMC Curves for a range of standard distance metrics')
plt.xlabel('Rank')
plt.ylabel('Recogniton Accuracy / %')
plt.legend(loc='best')

In [47]:
print (X_query.shape)
print (y_train.shape)

(1400, 2048)
(7368,)


In [ ]:
rank_accuracies_l_2 = []
mAP_l_2 = []
metric_l_2 = []

In [45]:

#Mahalanobis - inverse covariance


V = np.cov(X_train.T)

print (V.shape)

VI = np.linalg.inv(V)

print (VI.shape)



rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='mahalanobis',
                                       parameters = VI)

rank_accuracies_l_2.append(rank_accuracies)
mAP_l_2.append(mAP)
metric_l_2.append('Mahalanobis - Covariance')

(2048, 2048)
(2048, 2048)
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done
1 query image done


KeyboardInterrupt: 

In [ ]:
from metric_learn import MMC_Supervised

#Mahalanobis - learnt - reduced set

mmc = MMC_Supervised()
mmc.fit(X_train[0:100], y_train[0:100])



M = mmc.metric()


rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='mahalanobis',
                                       parameters = M)

rank_accuracies_l_2.append(rank_accuracies)
mAP_l_2.append(mAP)
metric_l_2.append('Learnt Mahalanobis (Red. Set)')


In [14]:
from metric_learn import MMC_Supervised

#Mahalanobis - learnt


mmc = MMC_Supervised()
mmc.fit(X_train, y_train)

KeyboardInterrupt: 

In [1]:

M = mmc.metric()


rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric ='mahalanobis',
                                       parameters = M)

rank_accuracies_l_2.append(rank_accuracies)
mAP_l_2.append(mAP)
metric_l_2.append('Learnt Mahalanobis')

NameError: name 'mmc' is not defined

In [ ]:
plt.figure(figsize=(8.0, 6.0))
color_list = ['green', 'blue', 'red', 'purple', 'orange', 'magenta', 'cyan', 'black', 'indianred', 'lightseagreen', 'gold', 'lightgreen']
for i in range(len(metric_l_2)):
    plt.plot(np.arange(1, 31), rank_accuracies_l_2[i], color=color_list[i], linestyle='dashed', label='Metric: '+ metric_l_2[i])

plt.title('CMC Curves for a range of standard distance metrics')
plt.xlabel('Rank')
plt.ylabel('Recogniton Accuracy / %')
plt.legend(loc='best')